#   Q18 Machine Learning Rolling Basis

In this example we predict whether the price will rise or fall by using supervised learning (Bayesian Ridge Regression). This template represents a starting point for developing a system which can take part to the **Q18 NASDAQ-100 Stock Long-Short contest**.

It consists of two parts.

* In the **first part** we just perform a global training of the time series using all time series data. We disregard the sequential aspect of the data and use also future data to train past data.

* In the **second part** we use the built-in backtester and perform training and prediction on a rolling basis in order to avoid forward looking. Please note that we are using a **specialized** version of the Quantiacs backtester which dramatically speeds up the the backtesting process by retraining your model on a regular basis.

**Features for learning**: we will use several technical indicators trying to capture different features. You can have a look at [**Technical Indicators**](https://quantiacs.com/documentation/en/user_guide/technical_indicators.html).

Please note that:

* Your trading algorithm can open short and long positions.

* At each point in time your algorithm can trade all or a subset of the stocks which at that point of time are or were part of the NASDAQ-100 stock index. Note that the composition of this set changes in time, and Quantiacs provides you with an appropriate filter function for selecting them.

* The Sharpe ratio of your system since January 1st, 2006, has to be larger than 1.

* Your system cannot be a copy of the current examples. We run a correlation filter on the submissions and detect duplicates.

* For simplicity we will use a single asset. It pays off to use more assets, ideally uncorrelated, and diversify your positions for a more solid Sharpe ratio.

More details on the rules can be found [here](https://quantiacs.com/contest).

**Need help?** Check the [**Documentation**](https://quantiacs.com/documentation/en/) and find solutions/report problems in the [**Forum**](https://quantiacs.com/community/categories) section.

**More help with Jupyter?** Check the official [**Jupyter**](https://jupyter.org/) page.

Once you are done, click on **Submit to the contest** and take part to our competitions.

API reference:

* **data**: check how to work with [data](https://quantiacs.com/documentation/en/reference/data_load_functions.html);

* **backtesting**: read how to run the [simulation](https://quantiacs.com/documentation/en/reference/evaluation.html) and check the results.

Need to use the optimizer function to automate tedious tasks?

* **optimization**: read more on our [article](https://quantiacs.com/community/topic/29/optimizing-and-monitoring-a-trading-system-with-quantiacs).

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) { return false; }
// disable widget scrolling

<IPython.core.display.Javascript object>

In [2]:
import logging

import xarray as xr  # xarray for data manipulation

import qnt.data as qndata     # functions for loading data
import qnt.backtester as qnbt # built-in backtester
import qnt.ta as qnta         # technical analysis library
import qnt.stats as qnstats   # statistical functions

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

np.seterr(divide = "ignore")

from qnt.ta.macd import macd
from qnt.ta.rsi  import rsi
from qnt.ta.stochastic import stochastic_k, stochastic, slow_stochastic

from sklearn import neighbors
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error

In [3]:
# load SP500 data
stock_data = qndata.stocks.load_spx_data(min_date="2007-06-01")

# Implementación de is_liquid******************************************************
is_liquid = stock_data.sel(field="is_liquid")

# assets list
assets = [
    "SPY:NVDA",  # NVIDIA Corporation
    "SPY:TSLA",  # Tesla Inc.
    "SPY:MSFT",  # Microsoft Corporation
    "SPY:AAPL",  # Apple Inc.
    "SPY:AMZN",  # Amazon.com Inc.
    "SPY:GOOGL", # Alphabet Inc. (Class A)
    "SPY:FB",    # Meta Platforms Inc.
    "SPY:BRK.B", # Berkshire Hathaway Inc. (Class B)
    "SPY:JNJ",   # Johnson & Johnson
    "SPY:V",     # Visa Inc.
    "SPY:PG",    # Procter & Gamble Co.
    "SPY:JPM",   # JPMorgan Chase & Co.
    "SPY:UNH",   # UnitedHealth Group Inc.
    "SPY:HD",    # Home Depot Inc.
    "SPY:MA",    # Mastercard Inc.
    "SPY:PFE",   # Pfizer Inc.
    "SPY:ABBV",  # AbbVie Inc.
    "SPY:MRK",   # Merck & Co. Inc.
    "SPY:PEP",   # PepsiCo Inc.
    "SPY:KO"     # The Coca-Cola Company
]  
ma_50 = stock_data.sel(field="close").rolling(time=50).mean().expand_dims('field').assign_coords(field=['ma_50'])
ma_200 = stock_data.sel(field="close").rolling(time=200).mean().expand_dims('field').assign_coords(field=['ma_200'])
volatility = stock_data.sel(field="close").rolling(time=50).std().expand_dims('field').assign_coords(field=['volatility'])

stock_data = xr.concat([stock_data, ma_50, ma_200, volatility], dim='field')

100% (367973 of 367973) |################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (79197 of 79197) |##################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (13084392 of 13084392) |############| Elapsed Time: 0:00:00 Time:  0:00:00


fetched chunk 1/12 1s


100% (13084392 of 13084392) |############| Elapsed Time: 0:00:00 Time:  0:00:00


fetched chunk 2/12 2s


100% (13084396 of 13084396) |############| Elapsed Time: 0:00:00 Time:  0:00:00


...

fetched chunk 10/12 11s


100% (13084360 of 13084360) |############| Elapsed Time: 0:00:00 Time:  0:00:00


fetched chunk 11/12 11s


100% (2858776 of 2858776) |##############| Elapsed Time: 0:00:00 Time:  0:00:00


fetched chunk 12/12 12s
Data loaded 12s


In [4]:
def get_features(data):
    # trend:
    trend = qnta.roc(qnta.lwma(data.sel(field="close"), 60), 1)

    # moving average convergence divergence (MACD):
    macd = qnta.macd(data.sel(field="close"))
    macd2_line, macd2_signal, macd2_hist = qnta.macd(data, 12, 26, 9)

    # volatility:
    volatility = qnta.tr(data.sel(field="high"), data.sel(field="low"), data.sel(field="close"))
    volatility = volatility / data.sel(field="close")
    volatility = qnta.lwma(volatility, 14)

    # the stochastic oscillator:
    k, d = qnta.stochastic(data.sel(field="high"), data.sel(field="low"), data.sel(field="close"), 14)

    # the relative strength index:
    rsi = qnta.rsi(data.sel(field="close"))

    # the logarithm of the closing price:
    price = data.sel(field="close").ffill("time").bfill("time").fillna(0)  # fill NaN
    price = np.log(price)

    # Momentum:
    momentum = qnta.roc(data.sel(field="close"), 14)

    # Bollinger Bands Width (manual calculation):
    close_price = data.sel(field="close")
    rolling_mean = close_price.rolling(time=20).mean()
    rolling_std = close_price.rolling(time=20).std()
    upper_band = rolling_mean + 2 * rolling_std
    lower_band = rolling_mean - 2 * rolling_std
    bb_width = (upper_band - lower_band) / close_price  # Normalized width

    # Combine all features:
    result = xr.concat(
        [
            trend, macd2_signal.sel(field="close"), volatility, d, rsi, price, 
            momentum, bb_width
        ],
        pd.Index(
            ["trend", "macd", "volatility", "stochastic_d", "rsi", "price", 
             "momentum", "bb_width"],
            name="field"
        )
    )

    return result.transpose("time", "field", "asset")

In [5]:
# displaying the features:
my_features = get_features(stock_data)
display(my_features.sel(field="trend").to_pandas())

asset,NAS:AAL,NAS:AAPL,NAS:ABNB,NAS:ACGL,NAS:ADBE,NAS:ADI,NAS:ADP,NAS:ADSK,NAS:AEP,NAS:AKAM,...,NYS:WMB,NYS:WMT,NYS:WRB,NYS:WST,NYS:WY,NYS:XOM,NYS:XYL,NYS:YUM,NYS:ZBH,NYS:ZTS
time,,,,,,,,,,,,,,,,,,,,,
2007-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-06-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-06-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-06-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-06-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-13,0.493014,-0.045787,-0.153089,-0.249322,-0.541151,-0.093185,-0.070384,-0.151030,-0.045134,-0.250878,...,0.067221,0.115991,-0.148073,0.196956,-0.211057,-0.167927,-0.225438,-0.264413,-0.060430,-0.173706
2025-01-14,0.595151,-0.061104,-0.182038,-0.199958,-0.503530,-0.064253,-0.053986,-0.128503,-0.043578,-0.210291,...,0.182261,0.083200,-0.076097,0.081556,-0.170299,-0.150235,-0.222408,-0.237264,-0.084761,-0.202293
2025-01-15,0.545674,0.000734,-0.058978,-0.135012,-0.463930,-0.006926,-0.035611,-0.083611,-0.018076,-0.205332,...,0.172389,0.097216,-0.001463,0.075304,-0.095950,-0.092666,-0.194745,-0.208297,-0.029387,-0.140074


In [6]:
def get_target_classes(data):
    """ Target classes for predicting if price goes up or down."""

    price_current = data.sel(field="close")
    price_future  = qnta.shift(price_current, -1)

    class_positive = 1 # prices goes up
    class_negative = 0 # price goes down

    target_price_up = xr.where(price_future > price_current, class_positive, class_negative)

    return target_price_up

In [7]:
# displaying the target classes:
my_targetclass = get_target_classes(stock_data)
display(my_targetclass.to_pandas())

asset,NAS:AAL,NAS:AAPL,NAS:ABNB,NAS:ACGL,NAS:ADBE,NAS:ADI,NAS:ADP,NAS:ADSK,NAS:AEP,NAS:AKAM,...,NYS:WMB,NYS:WMT,NYS:WRB,NYS:WST,NYS:WY,NYS:XOM,NYS:XYL,NYS:YUM,NYS:ZBH,NYS:ZTS
time,,,,,,,,,,,,,,,,,,,,,
2007-06-01,0,1,0,1,1,1,1,1,0,1,...,1,1,1,1,1,0,0,1,1,0
2007-06-04,0,1,0,0,0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2007-06-05,0,1,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2007-06-06,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2007-06-07,0,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-13,1,0,0,1,1,1,1,1,0,1,...,1,0,1,0,1,1,0,1,0,0
2025-01-14,0,1,1,1,1,1,1,1,1,0,...,0,1,1,1,1,1,1,1,1,1
2025-01-15,1,0,0,1,1,0,1,0,1,0,...,1,0,1,1,1,0,1,1,1,1


In [8]:
from sklearn.neighbors import KNeighborsRegressor

def get_model(n_neighbors=5):

    model = KNeighborsRegressor(n_neighbors=n_neighbors)
    type(model)
    print("model")
    return model

In [9]:
# Create and train the models working on an asset-by-asset basis.
# Create and train models on an asset-by-asset basis.
asset_name_all = stock_data.coords["asset"].values

models = dict()

total_assets = len(asset_name_all)  # Count the total number of assets

for idx, asset_name in enumerate(asset_name_all):
    print(f"Training model {idx + 1} of {total_assets}: {asset_name}")

    # Drop missing values:
    target_cur = my_targetclass.sel(asset=asset_name).dropna("time", how="any")
    features_cur = my_features.sel(asset=asset_name).dropna("time", how="any")

    # Filtrar activos líquidos*******************************************************
    is_liquid_cur = is_liquid.sel(asset=asset_name)
    target_cur = target_cur * is_liquid_cur
    features_cur = features_cur * is_liquid_cur

    # Align features and targets:
    target_for_learn_df, feature_for_learn_df = xr.align(target_cur, features_cur, join="inner")

    if len(features_cur.time) < 10:
        # Not enough points for training
        print(f"Not enough data for asset: {asset_name}. Skipping...")
        continue

    model = get_model()

    try:
        model.fit(feature_for_learn_df.values, target_for_learn_df)
        models[asset_name] = model
        print(f"Model trained for asset: {asset_name}")
    except Exception as e:
        logging.exception("Model training failed for asset: %s", asset_name)

print("Training completed for all models.")


Training model 1 of 516: NAS:AAL
model
Model trained for asset: NAS:AAL
Training model 2 of 516: NAS:AAPL
model
Model trained for asset: NAS:AAPL
Training model 3 of 516: NAS:ABNB
model
Model trained for asset: NAS:ABNB
Training model 4 of 516: NAS:ACGL
model
Model trained for asset: NAS:ACGL
Training model 5 of 516: NAS:ADBE
model
Model trained for asset: NAS:ADBE
Training model 6 of 516: NAS:ADI
model
Model trained for asset: NAS:ADI
Training model 7 of 516: NAS:ADP
model
Model trained for asset: NAS:ADP
Training model 8 of 516: NAS:ADSK
model
Model trained for asset: NAS:ADSK
Training model 9 of 516: NAS:AEP
model
Model trained for asset: NAS:AEP
Training model 10 of 516: NAS:AKAM
model
Model trained for asset: NAS:AKAM
Training model 11 of 516: NAS:ALGN
model
Model trained for asset: NAS:ALGN
Training model 12 of 516: NAS:AMAT
model
Model trained for asset: NAS:AMAT
Training model 13 of 516: NAS:AMD
model
Model trained for asset: NAS:AMD
Training model 14 of 516: NAS:AMGN
model
Mod

In [10]:
# Showing which features are more important in predicting:

importance = models["NAS:AAPL"].coef_
importance

for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

plt.bar([x for x in range(len(importance))], importance)
plt.show()

AttributeError: 'KNeighborsRegressor' object has no attribute 'coef_'

In [11]:
# Performs prediction and generates output weights:

asset_name_all = stock_data.coords["asset"].values
weights = xr.zeros_like(stock_data.sel(field="close"))

for asset_name in asset_name_all:
    if asset_name in models:
        model = models[asset_name]
        features_all = my_features
        features_cur = features_all.sel(asset=asset_name).dropna("time", "any")
        if len(features_cur.time) < 1:
            continue
        try:
            weights.loc[dict(asset=asset_name, time=features_cur.time.values)] = model.predict(features_cur.values)
        except KeyboardInterrupt as e:
            raise e
        except:
            logging.exception("model prediction failed")
# Filtrar predicciones para activos líquidos
weights = weights * is_liquid #**********************************************************************
    
print(weights)

/tmp/ipykernel_382/4091923526.py:10: FutureWarning:

Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.

/tmp/ipykernel_382/4091923526.py:10: FutureWarning:

Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.

/tmp/ipykernel_382/4091923526.py:10: FutureWarning:

Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.

/tmp/ipykernel_382/4091923526.py:10: FutureWarning:

Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.

/tmp/ipykernel_382/4091923526.py:10: FutureWarning:

Passing 'how' as positional argument(s) to dropna w

<xarray.DataArray 'stocks_s&p500' (time: 4438, asset: 516)> Size: 18MB
array([[nan, 0. , nan, ..., 0. , 0. , nan],
       [nan, 0. , nan, ..., 0. , 0. , nan],
       [nan, 0. , nan, ..., 0. , 0. , nan],
       ...,
       [0. , 0.2, 0.8, ..., 0.8, 0.4, 0.8],
       [0. , 0.2, 0.8, ..., 0.6, 0.8, 0.2],
       [0. , 0.4, 0.4, ..., 0.6, 0.4, 0.4]])
Coordinates:
  * time     (time) datetime64[ns] 36kB 2007-06-01 2007-06-04 ... 2025-01-17
  * asset    (asset) <U9 19kB 'NAS:AAL' 'NAS:AAPL' ... 'NYS:ZBH' 'NYS:ZTS'


/tmp/ipykernel_382/4091923526.py:10: FutureWarning:

Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.

/tmp/ipykernel_382/4091923526.py:10: FutureWarning:

Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.

/tmp/ipykernel_382/4091923526.py:10: FutureWarning:

Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.

/tmp/ipykernel_382/4091923526.py:10: FutureWarning:

Passing 'how' as positional argument(s) to dropna was deprecated in version v2023.10.0 and will raise an error two releases later. Please pass them as keyword arguments.



In [12]:
# Aplicar el filtro de activos líquidos a los pesos****************************************************
weights = weights * is_liquid

# Calcular el Sharpe ratio
sharpe = get_sharpe(stock_data, weights)
print(f"Sharpe Ratio: {sharpe}")


NameError: name 'get_sharpe' is not defined

In [ ]:
def get_sharpe(stock_data, weights):
    """Calculates the Sharpe ratio"""
    rr = qnstats.calc_relative_return(stock_data, weights)
    sharpe = qnstats.calc_sharpe_ratio_annualized(rr).values[-1]
    return sharpe

sharpe = get_sharpe(stock_data, weights)
sharpe

The sharpe ratio using the method above follows from **forward looking**. Predictions for (let us say) 2017 know about the relation between features and targets in 2020. Let us visualize the results:

In [13]:
import qnt.graph as qngraph
# Aplicar filtro en los pesos para activos líquidos
weights = weights * is_liquid

# Calcular estadísticas
statistics = qnstats.calc_stat(stock_data, weights)

# Mostrar estadísticas y gráficas
display(statistics.to_pandas().tail())

performance = statistics.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

# Mostrar Sharpe Ratio final
display(statistics[-1:].sel(field=["sharpe_ratio"]).transpose().to_pandas())

# Ver correlaciones con estrategias existentes
qnstats.print_correlation(weights, stock_data)


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2025-01-13,347.115985,0.008874,0.212571,-0.043539,-0.432111,1.855329,0.394390,1.0,515.0,0.460712,4.262757
2025-01-14,350.085627,0.008555,0.212554,-0.035356,-0.432111,1.858163,0.394960,1.0,515.0,0.460700,4.262731
2025-01-15,353.783952,0.010564,0.212541,-0.025165,-0.432111,1.861701,0.395689,1.0,515.0,0.460690,4.262768
2025-01-16,356.981852,0.009039,0.212525,-0.016354,-0.432111,1.864706,0.396297,1.0,515.0,0.460679,4.262826
2025-01-17,358.926383,0.005447,0.212504,-0.010996,-0.432111,1.866430,0.396623,1.0,515.0,0.460680,4.263486


time,2025-01-17
field,
sharpe_ratio,1.86643



Ok. This strategy does not correlate with other strategies.


In [ ]:
"""R2 (coefficient of determination) regression score function."""
r2_score(my_targetclass, weights, multioutput="variance_weighted")

In [ ]:
"""The explained variance score explains the dispersion of errors of a given dataset"""
explained_variance_score(my_targetclass, weights, multioutput="uniform_average")

In [ ]:
"""The explained variance score explains the dispersion of errors of a given dataset"""
mean_absolute_error(my_targetclass, weights)

Let us now use the Quantiacs **backtester** for avoiding **forward looking**.

The backtester performs some transformations: it trains the model on one slice of data (using only data from the past) and predicts the weights for the following slice on a rolling basis:

In [ ]:
def train_model(data):
    """Create and train the model working on an asset-by-asset basis."""

    asset_name_all = data.coords["asset"].values
    features_all   = get_features(data)
    target_all     = get_target_classes(data)

    models = dict()

    for asset_name in asset_name_all:

        # drop missing values:
        target_cur   = target_all.sel(asset=asset_name).dropna("time", "any")
        features_cur = features_all.sel(asset=asset_name).dropna("time", "any")

        target_for_learn_df, feature_for_learn_df = xr.align(target_cur, features_cur, join="inner")

        if len(features_cur.time) < 10:
                continue

        model = get_model()

        try:
            model.fit(feature_for_learn_df.values, target_for_learn_df)
            models[asset_name] = model

        except:
            logging.exception("model training failed")

    return models

In [ ]:
def predict_weights(models, data):
    """The model predicts if the price is going up or down.
       The prediction is performed for several days in order to speed up the evaluation."""

    asset_name_all = data.coords["asset"].values
    weights = xr.zeros_like(data.sel(field="close"))

    for asset_name in asset_name_all:
        if asset_name in models:
            model = models[asset_name]
            features_all = get_features(data)
            features_cur = features_all.sel(asset=asset_name).dropna("time", "any")

            if len(features_cur.time) < 1:
                continue

            try:
                weights.loc[dict(asset=asset_name, time=features_cur.time.values)] = model.predict(features_cur.values)

            except KeyboardInterrupt as e:
                raise e

            except:
                logging.exception("model prediction failed")

    return weights

In [ ]:
# Calculate weights using the backtester:
weights = qnbt.backtest_ml(
    train                         = train_model,
    predict                       = predict_weights,
    train_period                  =  2 *365,  # the data length for training in calendar days
    retrain_interval              = 10 *365,  # how often we have to retrain models (calendar days)
    retrain_interval_after_submit = 1,        # how often retrain models after submission during evaluation (calendar days)
    predict_each_day              = False,    # Is it necessary to call prediction for every day during backtesting?
                                              # Set it to True if you suspect that get_features is looking forward.
    competition_type              = "stocks_nasdaq100",  # competition type
    lookback_period               = 365,                 # how many calendar days are needed by the predict function to generate the output
    start_date                    = "2005-01-01",        # backtest start date
    analyze                       = True,
    build_plots                   = True  # do you need the chart?
)

The Sharpe ratio is obviously smaller as the training process is not looking forward (as it happens by processing data on a global basis), but performed on a rolling basis.

# May I import libraries?

Yes, please refer to the file **init.ipynb** in your home directory. You can for example use:

! conda install -y scikit-learn

# How to load data?

Daily stock data for the **Q18 Nasdaq-100** contest can be loaded using:
```python
data = qndata.stocks.load_ndx_data(tail = 17*365, dims = ("time", "field", "asset"))
```

Cryptocurrency daily data used for the Q16/Q17 contests can be loaded using:
```python
data = qndata.cryptodaily.load_data(tail = 17*365, dims = ("time", "field", "asset"))
```

Futures data for the Q15 contest can be loaded using:
```python
data= qndata.futures.load_data(tail = 17*365, dims = ("time", "field", "asset"))
```

BTC Futures data for the Q15 contest can be loaded using:
```python
data= qndata.cryptofutures.load_data(tail = 17*365, dims = ("time", "field", "asset"))
```

# How to view a list of all tickers?

```python
data.asset.to_pandas().to_list()
```

# How to see which fields are available?

```python
data.field.to_pandas().to_list()
```

# How to load specific tickers?

```python
data = qndata.stocks.load_ndx_data(tail=17 * 365, assets=["NAS:AAPL", "NAS:AMZN"])
```

# How to select specific tickers after loading all data?

```python
def get_data_filter(data, assets):
    filler= data.sel(asset=assets)
    return filler

get_data_filter(data, ["NAS:AAPL", "NAS:AMZN"])
```

# How to get the prices for the previous day?

```python
qnta.shift(data.sel(field="open"), periods=1)
```

or:

```python
data.sel(field="open").shift(time=1)
```

# How to get the Sharpe ratio?

```python
import qnt.stats as qnstats

def get_sharpe(market_data, weights):
    rr = qnstats.calc_relative_return(market_data, weights)
    sharpe = qnstats.calc_sharpe_ratio_annualized(rr).values[-1]
    return sharpe

sharpe = get_sharpe(data, weights) # weights.sel(time=slice("2006-01-01",None))
```

# How do I get a list of the top 3 assets ranked by Sharpe ratio?

```python
import qnt.stats as qnstats

data = qndata.stocks.load_ndx_data(tail = 17*365, dims = ("time", "field", "asset"))

def get_best_instruments(data, weights, top_size):
    # compute statistics:
    stats_per_asset = qnstats.calc_stat(data, weights, per_asset=True)
    # calculate ranks of assets by "sharpe_ratio":
    ranks = (-stats_per_asset.sel(field="sharpe_ratio")).rank("asset")
    # select top assets by rank "top_period" days ago:
    top_period = 1
    rank = ranks.isel(time=-top_period)
    top = rank.where(rank <= top_size).dropna("asset").asset

    # select top stats:
    top_stats = stats_per_asset.sel(asset=top.values)

    # print results:
    print("SR tail of the top assets:")
    display(top_stats.sel(field="sharpe_ratio").to_pandas().tail())

    print("avg SR = ", top_stats[-top_period:].sel(field="sharpe_ratio").mean("asset")[-1].item())
    display(top_stats)
    return top_stats.coords["asset"].values

get_best_instruments(data, weights, 3)
```

# How can I check the results for only the top 3 assets ranked by Sharpe ratio?

Select the top assets and then load their data:

```python
best_assets= get_best_instruments(data, weights, 3)

data= qndata.stocks.load_ndx_data(tail = 17*365, assets=best_assets)
```

# How can prices be processed?

Simply import standard libraries, for example **numpy**:

```python
import numpy as np

high= np.log(data.sel(field="high"))
```

# How can you reduce slippage impace when trading?

Just apply some technique to reduce turnover:

```python
def get_lower_slippage(weights, rolling_time=6):
    return weights.rolling({"time": rolling_time}).max()

improved_weights = get_lower_slippage(weights, rolling_time=6)
```

# How to use technical analysis indicators?

For available indicators see the source code of the library: /qnt/ta

## ATR

```python
def get_atr(data, days=14):
    high = data.sel(field="high") * 1.0
    low  = data.sel(field="low") * 1.0
    close= data.sel(field="close") * 1.0

    return qnta.atr(high, low, close, days)

atr= get_atr(data, days=14)
```

## EMA

```python
prices= data.sel(field="high")
prices_ema= qnta.ema(prices, 15)
```

## TRIX

```python
prices= data.sel(field="high")
prices_trix= qnta.trix(prices, 15)
```

## ADL and EMA

```python
adl= qnta.ad_line(data.sel(field="close")) * 1.0
adl_ema= qnta.ema(adl, 18)
```

# How can you check the quality of your strategy?

```python
import qnt.output as qnout
qnout.check(weights, data, "stocks_nasdaq100")
```

or

```python
stat= qnstats.calc_stat(data, weights)
display(stat.to_pandas().tail())
```

or

```python
import qnt.graph   as qngraph
statistics= qnstats.calc_stat(data, weights)
display(statistics.to_pandas().tail())

performance= statistics.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

display(statistics[-1:].sel(field = ["sharpe_ratio"]).transpose().to_pandas())
qnstats.print_correlation(weights, data)

```

# An example using pandas

One can work with pandas DataFrames at intermediate steps and at the end convert them to xarray data structures:

```python
def get_price_pct_change(prices):
    prices_pandas = prices.to_pandas()
    assets = data.coords["asset"].values
    for asset in assets:
        prices_pandas[asset] = prices_pandas[asset].pct_change()
    return prices_pandas

prices = data.sel(field="close") * 1.0
prices_pct_change = get_price_pct_change(prices).unstack().to_xarray()
```

# How to submit a strategy to the competition?

Check that weights are fine:

```python
import qnt.output as qnout
qnout.check(weights, data, "stocks_nasdaq100")
```

If everything is ok, write the weights to file:

```python
qnout.write(weights)
```

In your **personal account**:

* **choose** a strategy;
* click on the **Submit** button;
* select the type of competition.

At the beginning you will find the strategy under the **Checking** area:

* **Sent strategies** > **Checking**.

If technical checks are successful, the strategy will go under the **Candidates** area:

* **Sent strategies** > **Candidates**.

Otherwise it will be **Filtered**:

* **Sent strategies** > **Filtered**

and you should inspect error and warning messages.

Note that a strategy under the **Candidates** area should have a Sharpe ratio larger than 1 for being eligible for a prize. Please check warning messages in your **Candidates** area!